*The code snippet assumes Anaconda 5.2.0 version of Python virtual environment*

<div class="alert alert-info">
    <h4>Acknowledgement</h4>
    <p>The materials on this post are based the on five NLP papers, <a href="https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf" target="_blank">Distributed Representations of Words and Phrases and their Compositionality</a> (Mikolov et al., 2013), <a href="https://arxiv.org/pdf/1411.2738.pdf" target="_blank">word2vec Parameter Learning Explained</a> (Rong, 2014), <a href="https://arxiv.org/pdf/1710.09805.pdf" target="_blank">Improving Negative Sampling for Word Representation using Self-embedded Features</a> (Chen et al., 2017), <a href="https://aclweb.org/anthology/D17-1037" target="_blank">Incremental Skip-gram Model with Negative Sampling</a> (Kaji and Kobayashi, 2017), and <a href="https://arxiv.org/pdf/1402.3722.pdf" target="_blank">word2vec Explained: Deriving Mikolov et al.’s
Negative-Sampling Word-Embedding Method</a> (Goldberg and Levy, 2014).</p>
</div>

## Review on Word2Vec Skip-Gram

In my <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling" target="_blank">previous post</a>, I illustrated the neural network structure of Skip-Gram Word2Vec model that represents words in a vector space.

<div class="row" id="fig1">
    <div class="col"><img src="jupyter_images/word2vec_skip-gram.png" style="margin: 0;"></div>
    <div class="col-12"><p class="image-description">Figure 1: Neural network structure of Skip-Gram</p></div>
</div>

I also derived the cost function of Skip-Gram in <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#Derivation-of-Cost-Function" target="_blank">Derivation of Cost Function</a>:

<div id="eq-1" style="font-size: 1rem;">
$$
\begin{align}
  J(\theta) = & - \frac{1}{T} \sum_{t=1}^{T} \sum_{-c\leq j \leq c,j\neq 0} \log p(w_{t+j} \mid w_t ; \, \theta) \\ 
            
\end{align}
\tag{1} 
$$
</div>

where $p(w_{t+j} \mid w_t ; \, \theta)$ is a probability of observing $w_{t+j}$ given $w_{t}$ with parameters $\theta$. In vanilla Skip-Gram, the probability is computed with softmax. I also noted that <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#stochastic" target="_blank">stochastic gradient descent</a> (SGD) is used to mitigate computational burden — the size of $T$ in $\frac{1}{T} \sum^T_{t=1}$ can be billions or more in NLP applications. The new cost function using SGD is:

<div id="eq-2" style="font-size: 1rem;">
$$J(\theta; w^{(t)}) = -
\sum_{c=1}^{C}
log 
\frac{exp(W_{output_{(c)}} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)} \tag{2}$$
</div>

where $T$ is the size of training samples, $C$ is the <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#Window-Size-of-Skip-Gram" target="_blank">window size</a>, $V$ is the size of unique vocab in the corpus, and $W_{input}$, $W_{output}$ and $h$ are illustrated in <a href="#fig1">figure 1</a>. 

### Review on Softmax

Softmax is a multinomial regression classifier. It means that it classifies multiple labels, such as predicting if an hand-written digit is $0,\,1,\,2,\,...\,8\,$ or $9$. In case of binary classification (True or False), such as classifying fraud or not-fraud in bank transactions, binomial regression classifier called Sigmoid function is used.

In <a href="#eq-2">eq (2)</a>, the fraction inside the summation of log yields the probability distribution of all $V$-vocabs in the corpus, given the input word. In statistics, the conditional probability of $A$ given $B$ is denoted as $p(A|B)$. In Skip-Gram, we use the notation, $p(w_{context}| w_{center})$, to denote the conditional probability of observing a context word given a center word. It is obtained by using the softmax function:

<div id="eq-3" style="font-size: 1rem;">$$ p(w_{context}|w_{center}) = \frac{exp(W_{output_{(context)}} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)} \in \mathbb{R}^{1} \tag{3} $$</div>

Exponentiation ensures that the transformed values are positive, and the normalization factor in the denominator ensures that the values have a range of $[0, 1)$ and their sum equals $1$. 

<div id="fig2" class="row full_screen_margin mobile_responsive_plot_full_width">
    <div class="col"><img src="jupyter_images/softmax_function.png" style="height: 300px;"></div>
    <div class="col-12"><p class="image-description">Figure 2: softmax function transformation</p></div>
</div>

The probability is computed $V$ times using <a href="#eq-3">eq (3)</a> to obtain a conditional probability distribution of observing all $V$-unique vocabs in the corpus, given a center word ($w^{(t)}$). 

<div id="eq-4" style="font-size: 1rem;">$$ \left[ \begin{array}{c} p(w_{1}|w^{(t)}) \\ p(w_{2}|w^{(t)}) \\ p(w_{3}|w^{(t)}) \\ \vdots \\ p(w_{V}|w^{(t)}) \end{array} \right] = \frac{exp(W_{output} \cdot h)}{\sum^V_{i=1}exp(W_{output_{(i)}} \cdot h)} \in \mathbb{R}^{V}\tag{4} $$</div>

### Softmax is computationally very expensive

There is an issue with the vanilla Skip-Gram — softmax is computationally very expensive, as it requires scanning through the entire output embedding matrix ($W_{output}$) to compute the probability distribution of all $V$ words, where $V$ can be millions or more.

<div class="row full_screen_margin mobile_responsive_plot_full_width">
    <div class="col"><img src="jupyter_images/vanilla-skip-gram-complexity.png"></div>
    <div class="col-12"><p class="image-description">Figure 3: Algorithm complexity of vanilla Skip-Gram</p></div>
</div>

Furtheremore, the normalization factor in the denominator also requires $V$ iterations. In mathematical context, the normalization factor needs to be computed for each probability p($w_{context}| w_{center}$), making the alogrithm complexity = $O(V \times V)$. However, when implemented on code, the normalization factor is computed only once and cached as a Python variable, making the alogrithm complexity = $O(V + V) \approx O(V)$. This is possible because normalization factor is the same for all words.

Due to this computational inefficiency, **softmax is not used in most implementaions of Skip-Gram**. Instead we use an alternative called *negative sampling* with <a href="#sigmoid">sigmoid function</a>, which rephrases the problem into a set of independent binary classification task of algorithm complexity = $O(K \, + \, 1)$, where $K$ typically has a range of $[5, 20]$.

## Skip-Gram Negative Sampling

In Skip-Gram, assuming <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#stochastic" target="_blank">stochastic gradient descent</a>, weight marices in the neural network are updated for each training sample to correctly predict output. Let's assume that the training corpus has 10,000 unique vocabs ($V$ = 10000) and the hidden layer is 300-dimensional ($N$ = 300). This means that there are 3,000,000 neurons in the output weight matrix ($W_{output}$) that need to be updated for each training sample (Notes: for the input weight matrix ($W_{input}$), only 300 neurons are updated for each training sample. This is illustrated in *figure 18* of my <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#weight_update" target="_blank">previous post</a>.) Since the size of the training corpus ($T$) is very large, updating 3M neurons for each training sample is unrealistic in terms of computational efficiency. Negative sampling addresses this issue by updating only a small fraction of the output weight neurons for each training sample. 

In negative sampling, $K$ <a href="#neg_word">negative samples</a> are <a href="#neg_drawn">randomly drawn</a> from a <a href="#noise_dist">noise distribution</a>. $K$ is a hyper-parameter that can be empirically tuned, with a typical range of $[5,\, 20]$. For each training sample (positive pair: $w$ and $c_{pos}$), you randomly draw $K$ number of negative samples from a noise distribution $P_n(w)$, and the model will update $(K+1) \times N$ neurons in the output weight matrix ($W_{output}$). $N$ is the dimension of the hidden layer ($h$), or the size of a word vector. $+1$ accounts for a <a href="#pos_word">positive sample</a>. 

With the above assumption, if you set <code>K=9</code>, the model will update (9 + 1) * 300 = 3000 neurons, which is only 0.1% of the 3M neurons in $W_{output}$. This is computationally much cheaper than the original Skip-Gram, and yet maintains a good quality of word vectors.

The below figure has 3-dimensional hidden layer ($N=3$), 11 vocabs ($V=11$), and 3 negative samples ($K=3$). 

<div class="row" id="fig4" style="margin-top: 20px;">
    <div class="col"><img src="jupyter_images/neg_vs_skip.png" style="margin: 0;"></div>
    <div class="col-12"><p class="image-description">Figure 4: Skip-Gram model structure</p></div>
</div>

<div id="negsample" class="alert alert-info">
    <h4>Notes: Choice of $K$</h4>
    <p>The <a href="https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf" target="_blank">paper</a> (Mikolov et al., 2013) says that K=2 ~ 5 works for large data sets, and K=5 ~ 20 for small data sets.
    </p>
</div>

### How does negative sampling work?

 With negative sampling, word vectors are no longer learned by predicting context words of a center word. Instead of using softmax to compute the $V$-dimensional probability distribution of observing an output word given an input word, $p(w_O|w_I)$, the model uses sigmoid function to learn to differentiate the actual context words (*positive*) from randomly drawn words (*negative*) from the <a href="#noise_dist">noise distribution</a> $P_n(w)$. 
 
 Assume that the center word is *"regression"*. It is likely to observe *"regression"* + {*"logistic"*, *"machine"*, *"sigmoid"*, *"supervised"*, *"neural"*} pairs, but it is unlikely to observe *"regression"* + {*"zebra"*, *"pimples"*, *"Gangnam-Style"*, *"toothpaste"*, *"idiot"*}. The model maximizes the probability $p(D=1|w,c_{pos})$ of observing positive pairs, while minimizing the probability $p(D=1|w,c_{neg})$ of observing negative pairs. The idea is that if the model can distinguish between the positive pairs ($w$, $c_{pos}$) vs negative pairs ($w$, $c_{neg}$), good word vectors will be learned.
 
<div class="row full_screen_margin_md mobile_responsive_plot_full_width" id="fig5" style="margin-top: 20px;">
    <div class="col"><img src="jupyter_images/neg_binomial.png"></div>
    <div class="col-12"><p class="image-description">Figure 5: Binomial classification of negative sampling</p></div>
</div> 

Negative sampling converts multi-classification task into binary-classification task. The new objective is to predict, for any given word-context pair ($w,\,c$), whether the word ($c$) is in the context window of the the center word ($w$) or not. Since the goal is to identify a given word as True (*positive*, $D=1$) or False (*negative*, $D=0$), we use sigmoid function instead of softmax function. The probability of a word ($c$) appearing within the context of the center word ($w$) can be defined as the following:

<div id="eq-5" style="font-size: 1rem;">
$$
p(D=1|w,c;\theta)=\frac{1}{1+exp(-c_{output_{j}} \cdot w)}
\in \mathbb{R}^{1}
\tag{5} 
$$</div>

where $c$ is the word you want to know whether it came from the context window or the noise distribution. $w$ is the input (center) word, and $\theta$ is the <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#theta_in_cost" target="_blank">weight matrix</a> passed into the model. Note that $w$ is equivalent to the hidden layer ($h$). $c_{output_{(j)}}$ is the word vector from the output weight matrix ($W_{output}$) of <a href="#fig1">figure 1</a>. 

<a href="#eq-5">Eq (5)</a> computes the probability that the given word ($c$) is a <a href="#pos_word">positive word</a> ($D=1$). It only needs to be applied $K$ + 1 times instead of $V$ times for every word in the vocabulary, because $c_{output_{(j)}}$ comes from the concatenation of a true context word ($c_{pos}$) and $K$ <a href="#neg_word">negative words</a> ($W_{neg} = \{ c_{neg, j}|j=1,\cdots,K \}$):

<div id="eq-6" style="font-size: 1rem;">
$$
w_{output{(j)}} \in \{c_{pos}\} \cup W_{neg}
\tag{6} 
$$
</div>

This probability is computed $K + 1$ times to obtain a probability distribution of a true context word and $K$ negative samples:

<div id="eq-7" style="font-size: 1rem;">
$$
\left[ \begin{array}{c} p(D=1|w,c_{pos}) \\ p(D=1|w,c_{neg, 1}) \\ p(D=1|w,c_{neg, 2}) \\ p(D=1|w,c_{neg, 3}) \\ \vdots \\ p(D=1|w,c_{neg, K}) \end{array} \right] 
= 
\frac{1}{1+exp(-(\{c_{pos}\} \cup W_{neg}) \cdot h)}
\in \mathbb{R}^{K+1}\tag{7} 
$$
</div>

Compare this equation with <a href="#eq-4">eq (4)</a> — you will notice that <a href="#eq-7">eq (7)</a> is **computationally much cheaper because $K$ is between 5 ~ 20, whereas $V$ can be millions. Moreover, no extra iterations are necessary to compute the normalization factor** in the denominator of <a href="#eq-4">eq (4)</a>, because sigmoid function is a binary regression classifier. The algorithm complexity for probability distribution of vanilla Skip-Gram is $O(V)$, whereas negative sampling's is $O(K+1)$. This shows why negative sampling saves a significant amount of computational cost per iteration.

<div id="sigmoid" class="alert alert-info">
    <h4>Notes: Sigmoid function $\sigma(x)$</h4>
    <p>Sigmoid function is used for two-class logistic regression.</p>
    <p><div style="font-size: 1rem; margin-top: 20px;">$$ \sigma(x) = \frac{1}{1+exp(-x))}$$</div></p>
    <p>It is used to classify if a given sample is True or False based on the computed probability. The sample is classified as True if the value is greater then 0.5, and vice versa. For example, if you want to classify if a certain bank transaction is fraud or not, you will use sigmoid for binary classification.</p>
    <img class="admonition-image" style="border: 1px solid #ddd;" src="jupyter_images/sigmoid.png"/>
    <p>If you are working on a multi-class task, such as hand-written digit classification, you will use softmax regression classifier.</p>
</div>

<div style="display: none">
from matplotlib import pylab
import pylab as plt
import numpy as np
%matplotlib notebook

#sigmoid = lambda x: 1 / (1 + np.exp(-x))
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

mySamples = []
mySigmoid = []

x = plt.linspace(-10,10,10)
y = plt.linspace(-10,10,100)

plt.plot(y, sigmoid(y))
plt.title('Sigmoid Function $\sigma(x)$')
plt.text(4, 0.8, r'$\sigma(x)=\frac{1}{1+e^{-x}}$', fontsize=15)
plt.xlabel('X', fontsize=12)
plt.ylabel('$\sigma(x)$', fontsize=14)
</div>

<div><hr></div>
<div id="fig6" class="row give-margin-inline-big-plot mobile_responsive_plot_full_width">
    <div class="col"><img src="jupyter_images/negative_sample_text.png"></div>
    <div class="col-12"><p class="image-description">Figure 6: Choice of negative samples (Text source: <a href="https://petrowiki.org/Drilling_induced_formation_damage" target="_blank">Petrowiki</a>)</p></div>
</div>

For the purpose of illustration, consider the above paragraphs. Assume that our center word ($w$) is <code>drilling</code>, window size is $3$, and the number of negative samples ($K$) is $5$. With the window size of $3$, the contexts words are: *"engineer"*, *"traditionally"*, *"designs"*, *"fluids"*, *"with"*, and *"two"*. These context words are considered as positive labels ($D = 1$). Our current context word ($c_{pos}$) is <code>engineer</code>. We also need negative words. We randomly pick $5$-words from the <a href="#noise_dist">noise distribution</a> $P_n(w)$ of the corpus for each context word, and consider them as negative samples ($D = 0$). For the current context word, <code>engineer</code>, the 5 randomly drawn negative words ($c_{neg}$) are: *"minimized"*, *"primary"*, *"concerns"*, *"led"*, and *"page"*. 

The idea of negative sampling is that it is more likely to observe positive word pairs ($w$, $c_{pos}$) together than negative word pairs ($w$, $c_{neg}$) together in the corpus. The model attempts to maximize the the probability of observing positive pairs $p(c_{pos}|w) \rightarrow 1$ and minimize the probability of observing negative pairs $p(c_{neg}|w) \rightarrow 0$ simultaneously by iterating through the training samples and updating the weights ($\theta$). Note that the sum of the probability distribution obtained by sigmoid function (<a href="#eq-7">eq (7)</a>) does not need to equal $1$, unlike softmax (<a href="#eq-4">eq (4)</a>).

<div id="fig7" class="row">
    <div class="col"><img src="jupyter_images/neg_opt_1.png"></div>
    <div class="col-12"><p class="image-description">Figure 7: maximizing postive pairs and minimizing negative pairs</p></div>
</div>

By the time the output probability distribution is nearly one-hot-encoded as in $iter = 4$ of the above figure, weight matrices $\theta$ are optimized and good word vectors are learned. This optimization is achieved by maximizing the dot product of positive pairs ($c_{pos}\cdot w$) and minimizing the dot product of negative pairs ($c_{neg}\cdot w$) in <a href="#eq-18">eq (18)</a>. 

<div id="negsample" class="alert alert-info">
    <h4>Notes: Drawing random negative samples</h4>
    <p>For each positive word-context pair ($w,\,c_{pos}$), $K$ new negative samples are <a href="#neg_drawn">randomly drawn</a> from a <a href="#noise_dist">noise distribution</a>. In <a href="#fig6">figure 6</a>, there are 6 positive context words (<i>"engineer"</i>, <i>"traditionally"</i>, <i>"designs"</i>, <i>"fluids"</i>, <i>"with"</i>, and <i>"two"</i>) for one center word (<i>"drilling"</i>), and $K$ is 5. This means that a total of 6 * 5 = 30 word vectors are updated for each center word $w$.</p>
</div>

<div id="pos_word"></div>
#### What is a positive word $c_{pos}$?

Words that actually appear within the context window of the center word ($w$). After the model is optimized, the probability computed with <a href="#eq-5">eq (5)</a> for positive words $c_{pos}$ will output $\approx$ 1 as shown in <a href="#fig7">fig 7</a>. A word vector of a center word ($w$) will be more similar to a word vector of positive word ($c_{pos}$) than of randomly drawn negative words ($c_{neg}$). This is because words that frequently appear together show strong correlation with each other. Therefore, once the model is optimized: $p(D=1|w,c_{pos})\approx1$.

<div id="neg_word"></div>
#### What is a negative word $c_{neg}$?

Words that are randomly drawn from a noise distribution $P_n(w)$. After the model is optimized, the probability computed with <a href="#eq-5">eq (5)</a> for negative words $c_{neg}$ will output $\approx$ 0 as shown in <a href="#fig7">fig 7</a>. When training an Word2Vec model, the vocab size ($V$) easily exceeds tens of thousands. When 5 ~ 20 negative samples are randomly drawn among the vocabs, it is unlikely to observe the random word with a center word together in the corpus. Therefore, once the model is optimized: $p(D=1|w,c_{neg})\approx0$.

<div id="noise_dist"></div>

#### What is a noise distribution $P_n(w)$?

Imagine a distribution of words based on how many times each word appeared in a corpus, denoted as $U(w)$ (this is called unigram distribution). For each word $w$, divide the number of times it appeared in a corpus by a normalization factor $Z$ so that the distribution becomes a probability distribution of range $[0, 1)$ and sums up to $1$. Raise the normalized distribution to the power of $\alpha$ so that the distribution is "smoothed-out". Then this becomes your noise distribution $P_n(w)$ — normalized frequency distribution of words raised to the power of $\alpha$. Mathematically, it can be expressed as:

<div id="eq-8" style="font-size: 1rem;">
$$
P_n(w) = \left(\frac{U(w)}{Z}\right)^{\alpha}
\tag{8} 
$$</div>

Raising the unigram distribution $U(w)$ to the power of $\alpha$ has an effect of smoothing out the distribution. **It attempts to combat the imbalance between common words and rare words** by decreasing the probability of drawing common words, and increasing the probability drawing rare words. 

$\alpha$ is a hyper-parameter that can be empirically tuned. The authors of the original Word2Vec <a href="https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf" target="_blank">paper</a> claims that the unigram distribution $U(w)$ raised to the $3/4$rd power (i.e., $U(w)^{3/4}/Z$) yielded the best result.

<div id="fig8" class="row">
    <div class="col"><img src="jupyter_images/noise_dist.png"></div>
    <div class="col-12"><p class="image-description">Figure 8: Effect of raising power of unigram distribution $U(w)$</p></div>
</div>

<div id="incremental_noise_dist" class="alert alert-info">
    <h4>Notes: Incremental noise distribution</h4>
    <p>Noise distribution $P_n(w)$ needs to be pre-computed by iterating through the entire corpus to obtain word frequency $U(w)$ and normalization factor $Z$ of the distribution. If an additional training data is added to the corpus, $P_n(w)$ needs to be computed all over again. To address this problem, a simple incremental extension of negative sampling was provided in this <a href="https://aclweb.org/anthology/D17-1037" target="_blank">paper</a> (Kaji and Kobayashi, 2017).</p>
</div>

<div style="display: none">
    
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib notebook

# sample data generation
data = sorted(stats.norm.rvs(size=1000) + 5)

# fit normal distribution
mean, std = stats.norm.fit(data, loc=0)
pdf_norm = stats.norm.pdf(data, mean, std)

temp = np.power(data, 3/4)
temp_mean, temp_std = stats.norm.fit(temp, loc=0)
temp_pdf_norm = stats.norm.pdf(temp, temp_mean, temp_std)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharey=True)
ax1.set_title('$U(w)$')
ax1.hist(temp, bins='auto', density=True)
ax1.set_xticklabels(['apple', 'desk', 'cup', 'chair', 'zebra', 'room', 'pencil', 'water', 'coin'])
ax1.plot(temp, temp_pdf_norm, label='norm')
ax2.set_title('$U(w)^{3/4}$')
ax2.hist(data, bins='auto', density=True)
ax2.plot(data, pdf_norm, label='norm')
ax2.set_xticklabels(['apple', 'desk', 'cup', 'chair', 'zebra', 'room', 'pencil', 'water', 'coin'])
</div>

<div id="neg_drawn"></div>
#### How are negative samples drawn?

$K$-negative samples are randomly drawn from a noise distribution $P_n(w)$. The noise distribution is generated with <a href="#eq-8">eq (8)</a> and the random samples are drawn with <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html" target="_blank">np.random.choice</a>.

In [2]:
import numpy as np

In [7]:
unig_dist  = {'apple': 0.023, 'bee': 0.12, 'desk': 0.34, 'chair': 0.517}

sum(unig_dist.values())

1.0

In [5]:
alpha      = 3 / 4

noise_dist = {key: val ** alpha for key, val in unig_dist.items()}
Z = sum(noise_dist.values())
noise_dist_normalized = {key: val / Z for key, val in noise_dist.items()}
noise_dist_normalized

{'apple': 0.044813853132981724,
 'bee': 0.15470428538870049,
 'desk': 0.33785130228003507,
 'chair': 0.4626305591982827}

In [8]:
sum(noise_dist_normalized.values())

1.0

<div style="margin-top: -25px;"><hr></div>
In the initial unigram distribution, <code>chair</code> appeared the most in the corpus, and had <code>0.517</code> chance of being drawn as a negative sample. However, the unigram distribution is raised to the power of $3/4$rd to combat the imbalance between common vs rare words, as shown in <a href="#fig8">figure 8</a>. After <code>unig_dist</code> was raised to the power of <code>alpha = 3/4</code> and normalized, <code>chair</code> now has <code>0.463</code> chance of being drawn. 

On the other hand, <code>apple</code> had the lowest probability (<code>0.023</code>) of being drawn. After the transformation, it now has a bit higher probability (<code>0.049</code>) of being drawn. The imbalance between the most common word (<code>chair</code>) and the least common word (<code>apple</code>) was mitigated. 

Once the noise distribution (<code>noise_dist_normalized</code>) is generated, you randomly draw $K$ negative samples according to each word's probability.

In [6]:
K = 10

np.random.choice(list(noise_dist_normalized.keys()), size=K, p=list(noise_dist_normalized.values()))

array(['apple', 'chair', 'bee', 'desk', 'chair', 'bee', 'bee', 'chair',
       'desk', 'chair'], dtype='<U5')

## Derivation of Cost Function in Negative Sampling

The derivations written here are based on the work of <a href="https://arxiv.org/pdf/1402.3722.pdf" target="_blank">word2vec Explained: Deriving Mikolov et al.’s
Negative-Sampling Word-Embedding Method</a> (Goldberg and Levy, 2014).

Consider a pair ($w,\,c$) of center word and its context. Did this pair come from the context window or the <a href="#noise_dist">noise distribution</a>? Let $p(D=1|w,c)$ be the probability that ($w,\,c$) is observed in the true corpus, and $p(D=0|w,c) = 1 - p(D=1|w,c)$ the probability that ($w,\,c$) is non-observed. There are <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#theta_in_cost" target="_blank">parameters</a> $\theta$ controlling the probability distribution: $p(D=1|w,c;\theta)$.

Negative sampling attempts to optimize the parameters $\theta$ by maximizing the probability of observing positive pairs ($w,\,c_{pos}$) while minimizing the probability of observing negative pairs ($w,\,c_{neg}$). For each positive pair, the model randomly draws $K$ negative words $W_{neg} = \{ c_{neg, j}|j=1,\cdots,K \}$. Our objective function is then:

<div id="eq-9" style="font-size: 1rem;">
$$
\begin{align}
     & \underset{\theta}{\text{argmax}} \ p(D=1|w,c_{pos};\theta) \prod_{c_{neg} \in W_{neg}} p(D=0|w,c_{neg};\theta) \label{}\tag{9}\\ 
   =\quad & \underset{\theta}{\text{argmax}} \ p(D=1|w,c_{pos};\theta) \prod_{c_{neg} \in W_{neg}} (1 - p(D=1|w,c_{pos};\theta))\label{}\tag{10}
\end{align}
$$</div>

<div class="alert alert-info">
    <h4>Notes: Probability Product</h4>
    <p>In statistics, probability of observing $C$ multiple events at the same time is computed by the product of each event's probability.</p>
    <p><center style="font-size: 1rem; margin-top: 20px;">$$p(x_{1}, x_{2} ... x_{C}) = p(x_{1}) \times p(x_{2}) \, \times \, ... \, \times \, p(x_{C})$$</center></p>
    <p>This can be shortened with a product notation:</p>
    <p><center style="font-size: 1rem; margin-top: 20px;">$$p(x_{1}, x_{2} ... x_{C}) = \prod_{c=1}^{C}p(x_{c})$$</center></p>
</div>

It is a common practice in machine learning to <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#take_natural_log" target="_blank">take a natural log to the objective function</a> to simplify derivations. This does not affect the optimized weights ($\theta$) because natural log is a monotonically increasing function. It ensures that the maximum value of the original probability function occurs at the same point as the log probability function. Therefore:

<div id="eq-11" style="font-size: 1rem;">
$$
\begin{align}
   =\quad & \underset{\theta}{\text{argmax}} \ log \, (p(D=1|w,c_{pos};\theta) \prod_{c_{neg} \in W_{neg}} p(D=0|w,c_{neg};\theta)) \label{}\tag{11}
\end{align}
$$</div>

Using the property of logs, the objective function can be simplified:

<div id="eq-12" style="font-size: 1rem;">
$$
\begin{align}
   =\quad  & \underset{\theta}{\text{argmax}} \ log \, p(D=1|w,c_{pos};\theta) + log \, \prod_{c_{neg} \in W_{neg}} p(D=0|w,c_{neg};\theta) \label{}\tag{12}\\ 
   =\quad & \underset{\theta}{\text{argmax}} \ log \, p(D=1|w,c_{pos};\theta) + \sum_{c_{neg} \in W_{neg}} log \, (1 - p(D=1|w,c_{neg};\theta)) \label{}\tag{13}
\end{align} 
$$</div>

The binomial probability $p(D=1|w,c;\theta)$ can be replaced with <a href="#eq-5">eq (5)</a>:

<div id="eq-14" style="font-size: 1rem;">
$$
\begin{align}
   =\quad  & \underset{\theta}{\text{argmax}} \ log \, \frac{1}{1+exp(-c_{pos} \cdot w)} + \sum_{c_{pos} \in W_{neg}} log \, (1 - \frac{1}{1+exp(-c_{neg} \cdot w)}) \label{}\tag{14}\\ 
   =\quad  & \underset{\theta}{\text{argmax}} \ log \, \frac{1}{1+exp(-c_{pos} \cdot w)} + \sum_{c_{pos} \in W_{neg}} log \, \frac{1}{1+exp(c_{neg} \cdot w)} \label{}\tag{15}\\ 
\end{align}
$$</div>

Using the definition of <a href="#sigmoid">sigmoid</a> function $\sigma(x)=\frac{1}{1+exp(-x)}$:

<div id="eq-16" style="font-size: 1rem;">
$$
\begin{align}
   =\quad  & \underset{\theta}{\text{argmax}} \ log \, \sigma(c_{pos} \cdot w) + \sum_{c_{pos} \in W_{neg}} log \, \sigma(-c_{neg} \cdot w) \label{}\tag{16}  
\end{align} 
$$</div>

According to Mikolov, <a href="#eq-16">eq (16)</a> replaces every $log \, p(w_O|w_I)$ in the vanilla Skip-Gram cost function defined in <a href="#eq-1">eq (1)</a>. Then, the cost function we want to minimize becomes: 

<div id="eq-17" style="font-size: 1rem;">$$J(\theta) = - \frac{1}{T} \sum_{i = 1}^T \sum_{-c\leq j \leq c,j\neq 0}( log \, \sigma(c_{pos} \cdot w) + \sum_{c_{neg} \in W_{neg}} log \, \sigma(-c_{neg} \cdot w)) \tag{17}$$</div>

However, when implemented in codes, batch gradient descent (making one update after iterating through the entire $T$ corpus) is almost never used due to its high computational cost. Instead, we use <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#stochastic" target="_blank">stochastic gradient descent</a>. Also, for negative sampling, gradients are calculated and weights are updated for each positive training pairs ($w,\, c_{pos}$). The new cost function is then:

<div id="eq-18" style="font-size: 1rem;">$$J(\theta; w,c_{pos}) = - log \, \sigma(c_{pos} \cdot w) - \sum_{c_{neg} \in W_{neg}} log \, \sigma(-c_{neg} \cdot w) \tag{18}$$</div>

### Derivation of gradients

The goal of any machine learning model is to find the optimal values of a weight matrix ($\theta$) to minimize prediction error. A general update equation for weight matrix looks like the following: 

<div style="font-size: 1rem;" id="eq-19">$$ \theta^{(new)}=\theta^{(old)}-\eta\cdot\frac{\partial J}{\partial \theta} \tag{19}$$</div>

$\theta$ is a parameter that needs to be optimized, and $\eta$ is a learning rate. In negative sampling, we take the derivative to the cost function $J(\theta; w,c_{pos})$ defined in <a href="#eq-18">eq (18)</a> with respect to $\theta$. Note that the derivative of a sigmoid function is $\frac{\partial \sigma}{\partial x} = \sigma(x)(1–\sigma(x))$.

<div style="font-size: 1rem;" id="eq-20">$$ \frac{\partial J}{\partial \theta} = (\sigma(c_{pos} \cdot w)) - 1)\frac{\partial c_{pos} \cdot w}{\partial \theta} + \sum_{c_{neg} \in W_{neg}} \sigma(c_{neg} \cdot w) \frac{\partial c_{neg} \cdot w}{\partial \theta} \tag{20}$$</div>

Since the parameter $\theta$ is a <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#theta_in_cost">concatenation</a> of the input and output weight matrix $[W_{input} \quad W_{output}]$, the cost function needs to be differentiated with respect to the both matrices — $[\frac{\partial J}{\partial W_{input}} \quad \frac{\partial J}{\partial W_{output}}]$.

<div class="alert alert-info">
    <h4>Notes: Clarification on notation</h4>
    <p>$w$ is a word vector for the input (center) word. It is equivalent to the <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#hidden_layer" target="_blank">hidden layer</a> $h$. I used the notation $w$ instead of $h$ to stick to the notation used in the <a href="https://arxiv.org/pdf/1402.3722.pdf" target="_blank">paper</a> (Goldberg and Levy, 2014).</p>
    <p>$c$ represents a word vector for context words. It can come from either the true context window ($c_{pos}$) or <a href="#noise_dist">noise distribution</a> ($c_{neg}$).</p>
</div>

<div id="grad_W_output"></div>

**Gradients with respect to output weight matrix $\frac{\partial J}{\partial W_{output}}$**

With negative sampling, we do not update the entire output weight matrix $W_{output}$, but only a fraction of it. We update $K + 1$ word vectors in the output weight matrix — $c_{pos}$, $c_{neg,1},\,...\,c_{neg,K}$. We take partial derivatives to the cost function defined in <a href="#eq-18">eq (18)</a> with respect to positive words and negative words. This can be done by replacing $\theta$ in <a href="#eq-20">eq (20)</a> with $c_{pos}$ and $c_{neg}$ each.

<div id="eq-21" style="font-size: 1rem;">
$$
\begin{align}
   \frac{\partial J}{\partial c_{pos}} &= (\sigma(c_{pos} \cdot w) - 1)\cdot w \label{}\tag{21}\\[5pt]
   \frac{\partial J}{\partial c_{eng}} &= \sigma(c_{neg} \cdot w)\cdot w \label{}\tag{22}
\end{align}
$$</div>

The update equations are then:

<div id="eq-23" style="font-size: 1rem;">
$$
\begin{align}
   c_{pos}^{(new)} &= c_{pos}^{(old)} - \eta \cdot (\sigma(c_{pos} \cdot w) - 1)\cdot w \label{}\tag{23}\\[6pt]
   c_{neg}^{(new)} &= c_{neg}^{(old)} - \eta \cdot \sigma(c_{pos} \cdot w) \label{}\tag{24}
\end{align}
$$</div>

The gradients for positive and negative words can be merged for brevity:

<div style="font-size: 1rem;" id="eq-25">$$ v_{output}^{(new)}=v_{output}^{(old)}-\eta\cdot (\sigma(v_{output} \cdot w) - t_j) \cdot w \tag{25}$$</div>

where $t_j = 1$ for positive words ($c_{pos}$) and $t_j = 0$ for negative words ($c_{pos} \in W_{neg}$). $v_{output}$ is a word vector in the output word matrix $W_{output}$. For each positive pairs ($w$, $c_{pos}$), <a href="#eq-25">eq (25)</a> is applied to $K + 1$ word vectors in $W_{output}$ as shown in <a href="#fig4">figure 4</a>.

<div id="grad_W_input"></div>

**Gradients with respect to input weight matrix $\frac{\partial J}{\partial W_{input}}$**

Just like vanilla Skip-Gram, only one word vector that corresponds to the input word $w$ in $W_{input}$ is updated with negative sampling. This is because the <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#input_layer" target="_blank">input layer is one-hot-encoded vector</a>. However, the equation for input word vector gradient is different.

<div id="eq-26" style="font-size: 1rem;">
$$
\begin{align}
   \frac{\partial J}{\partial w} &= (\sigma(c_{pos} \cdot w) - 1) \cdot c_{pos} + \sum_{c_{eng} \in W_{neg}} \sigma(c_{neg} \cdot w) \cdot c_{neg} \label{}\tag{26}\\
   &= \sum_{w_j \in \{c_{pos}\} \cup W_{neg}} (\sigma(c_{j} \cdot w) - t_j) \cdot c_{j} \label{}\tag{27}
\end{align}
$$</div>

Same as <a href="#eq-25">eq (25)</a>, $t_j = 1$ for positive words ($c_{pos}$) and $t_j = 0$ for negative words ($c_{pog} \in W_{neg}$). The update equation is then:

<div style="font-size: 1rem;" id="eq-28">$$ w_{input}^{(new)}=w_{input}^{(old)}-\eta\cdot \sum_{w_j \in \{c_{pos}\} \cup W_{neg}} (\sigma(c_{j} \cdot w) - t_j) \cdot c_{j} \tag{28}$$</div>

<div id="algorithm"></div>
## Negative Sampling Algorithm

For each positive word-context pair ($w, c_{pos}$), $1$ word vector that corresponds to the center word ($w$) is updated in the input weight matrix $W_{input}$ (as shown in *figure 18* of my <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#weight_update" target="_blank">previous post</a>). In the output weight matrix $W_{output}$, $1+K$ word vectors that correspond to the positive ($c_{pos}$) and negative ($c_{neg}$) words are updated as shown in <a href="#fig4">figure 4</a>. 

Since Skip-Gram uses SGD to reduce computational cost, negative sampling also uses SGD too. Then, the training for Skip-Gram negative sampling has the following algorithm structure:

<div id="sgns_algorithm"></div>
<p><u>Algorithm 1: Skip-Gram Negative Sampling</u></p>
<pre>
    <code class="language-python">
        P_nw = # generate noise distribution
        for word in corpus:
            for context in context_window:

                # draw K negative samples from P_nw
                W_neg = np.random.choice(Pn_w.keys(), size=K, p=Pn_w.values())

                # compute gradients. w is a hidden layer = input word vector
                grad_V_output_pos = (sigmoid(c_pos * w) - 1) * w
                grad_V_input = (sigmoid(c_pos * w) - 1) * c_pos
                grad_V_output_neg_list = []
                for c_neg in W_neg:
                    grad_V_output_neg_list.append(sigmoid(c_neg * w) * w)
                    grad_V_input += sigmoid(c_neg * w) * c_neg

                # use SGD to update w, c_pos, and c_neg_1, ... , c_neg_K
                V_output_pos = V_output_pos - alpha * grad_V_output_pos
                V_input = V_input - alpha * grad_V_input
                for grad_V_output_neg in grad_V_output_neg_list:
                    V_output_neg = V_output_neg - alpha * grad_V_output_neg
    </code>
</pre>


In vanilla Skip-Gram, one update is made for the entire weight matrices $[W_{input} \quad W_{output}]$ for each input word. Each update involves summing up dot products for all context words within the context window of size $C$ as shown in eq (20) and eq (21) of my <a href="https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling#eq-20" target="_blank">previous post</a>. 

In negative sampling, $C$ updates are made for a fraction of the weights for each input word. This is because negative sampling treats each positive pair ($w$, $c_{pos}$) as one training sample, whereas vanilla Skip-Gram treats a center word ($w$) and its $C$ neighboring context words ($c_{pos, 1}$, $...$, $c_{pos,C}$) all together as one training sample for SGD.

## Numerical Demonstration

Coming soon...